In [ ]:
import ipywidgets as widgets
from ipywidgets import interact
from interactive_func import plot_sync_async
x_widget=widgets.IntRangeSlider(
    value=[864, 1728],
    min=0,
    max=2000,
    step=72,
    description='Test:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d',
)

fre_opt=widgets.BoundedIntText(
    value=2,
    min=1,
    max=100,
    step=1,
    description='Frequancy: 1/',
    disabled=False
)
interact(plot_sync_async, fre=fre_opt, nodes=x_widget)

In [ ]:
import ipywidgets as widgets
from ipywidgets import interact
from interactive_single import interactive_insitu
log_button = widgets.ToggleButtons(
    options=['Synchronous', 'Asynchronous', 'Hybrid', 'Auto'],
    description='Aproaches:',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltips=['sync', 'async', 'hybrid'],
#     icons=['check'] * 3
)

y_widget = widgets.IntSlider(min=72, max=1728, step=72, value=864, description = "total_core")
x_widget = widgets.IntSlider(min=1, max=1728, step=1, value=864, description = " ")

def update_x_range(*args):
    if log_button.value == "Asynchronous" : 
        x_widget.max = y_widget.value - 1
        x_widget.description = "app_core"
        x_widget.value = y_widget.value / 2
    elif log_button.value == "Synchronous" : 
        x_widget.value = y_widget.value
        x_widget.max = y_widget.max
        x_widget.description = " "

y_widget.observe(update_x_range, 'value')
log_button.observe(update_x_range, 'value')
fre_opt=widgets.BoundedIntText(
    value=2,
    min=1,
    max=100,
    step=1,
    description='Frequancy: 1/',
    disabled=False
)
interact(interactive_insitu, total_core=y_widget, app_core=x_widget, freq=fre_opt, method=log_button)

In [ ]:
import ipywidgets as widgets
from ipywidgets import interact
from interactive_double import interactive_insitu
log_button = widgets.ToggleButtons(
    options=['Synchronous', 'Asynchronous', 'Hybrid','Auto'],
    description='Aproaches:',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltips=['sync', 'async', 'hybrid'],
#     icons=['check'] * 3
)

y_widget = widgets.IntSlider(min=4, max=28, step=4, value=8, description = "gpu")
x_widget = widgets.IntSlider(min=1, max=18, step=1, value=3, description = "cores / gpu")
z_widget = widgets.IntSlider(min=1, max=504, step=1, value=8, description = "in-situ core")
def update_x_range(*args):
    if log_button.value == "Hybrid" : 
        z_widget.max = y_widget.value * (18 - x_widget.value)
        x_widget.max = 17
        z_widget.min = y_widget.value / 4
        z_widget.step = y_widget.value / 4
        z_widget.description = "in-situ core"
    elif log_button.value == "Synchronous" : 
        z_widget.value = y_widget.value * x_widget.value
        z_widget.max = y_widget.value * 18 
        x_widget.max = 18
        z_widget.min = 1
        z_widget.step = 1
        z_widget.description = "in-situ core"
    elif log_button.value == "Auto" : 
        z_widget.max = y_widget.value * (18 - x_widget.value)
        x_widget.max = 17
        z_widget.min = y_widget.value / 4
        z_widget.step = y_widget.value / 4
        z_widget.value = 9
        z_widget.description = " "
        
y_widget.observe(update_x_range, 'value')
x_widget.observe(update_x_range, 'value')
log_button.observe(update_x_range, 'value')
fre_opt=widgets.BoundedIntText(
    value=10,
    min=1,
    max=100,
    step=1,
    description='Frequancy: 1/',
    disabled=False
)
interact(interactive_insitu, gpu=y_widget, ppg=x_widget, core=z_widget, freq=fre_opt, method=log_button)

In [1]:
import ipywidgets as widgets
from ipywidgets import interact
from interactive_qe import interactive_insitu
log_button = widgets.ToggleButtons(
    options=['Synchronous', 'Asynchronous', 'Hybrid','Auto'],
    description='Aproaches:',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltips=['sync', 'async', 'hybrid'],
#     icons=['check'] * 3
)

y_widget = widgets.IntSlider(min=4, max=28, step=4, value=8, description = "gpu")
x_widget = widgets.IntSlider(min=1, max=18, step=1, value=3, description = "ranks / gpu")
x2_widget = widgets.IntSlider(min=1, max=18, step=1, value=2, description = "threads / rank")
z_widget = widgets.IntSlider(min=1, max=504, step=1, value=8, description = "in-situ core")
def update_x_range(*args):
    if log_button.value == "Asynchronous" : 
        z_widget.max = y_widget.value * (18 - x_widget.value*x2_widget.value)
        x_widget.max = 18/x2_widget.value -1
        z_widget.min = y_widget.value / 4
        z_widget.step = y_widget.value / 4
        z_widget.description = "in-situ core"
    elif log_button.value == "Synchronous" : 
        z_widget.value = y_widget.value * x_widget.value
        z_widget.max = y_widget.value * 18 
        x_widget.max = 18
        z_widget.min = 1
        z_widget.step = 1
        z_widget.description = "in-situ core"
    elif log_button.value == "Auto" : 
        #z_widget.max = y_widget.value * (18 - x_widget.value*x2_widget.value)
        x_widget.max = 17
        x2_widget.max = 18 / x_widget.value
        z_widget.min = y_widget.value / 4
        z_widget.step = y_widget.value / 4
        z_widget.description = " "
        
y_widget.observe(update_x_range, 'value')
x_widget.observe(update_x_range, 'value')
x2_widget.observe(update_x_range, 'value')
log_button.observe(update_x_range, 'value')
fre_opt=widgets.BoundedIntText(
    value=10,
    min=1,
    max=1000,
    step=1,
    description='Frequancy: 1/',
    disabled=False
)
interact(interactive_insitu, gpu=y_widget, ppg=x_widget, r=x2_widget, core=z_widget, freq=fre_opt, method=log_button)

interactive(children=(IntSlider(value=8, description='gpu', max=28, min=4, step=4), IntSlider(value=3, descrip…

<function interactive_qe.interactive_insitu(gpu, ppg, r, core, freq, method)>